In [9]:
import matplotlib.pyplot as plt
import pickle
from os import walk
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
path = "./UTKFace"

In [ ]:
filenames = next(walk(path), (None, None, []))[2]
dataset = []
for f in filenames:
  print(f)
  dataset.append((plt.imread("./UTKFace/"+f), f))

#Serializzazione
with open("dataset.pickle", "wb") as outfile:
 pickle.dump(dataset, outfile)

In [ ]:
with open("dataset.pickle", "rb") as infile:
    test_dataset_reconstructed = pickle.load(infile)
print("Oggetto Ricostruito")

def compare_lists_of_tuples(list1, list2):
    if len(list1) != len(list2):
        return False
    for (mat1, label1), (mat2, label2) in zip(list1, list2):
        if not np.array_equal(mat1, mat2) or label1 != label2:
            return False
    return True

# Confronto delle liste
if compare_lists_of_tuples(dataset, test_dataset_reconstructed):
    print("Le liste di tuple sono uguali.")
else:
    print("Le liste di tuple sono diverse.")

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=1)